In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted
import matplotlib.pyplot as plt
import numpy as np

In [ ]:

class BundleReducer(BaseEstimator):
    def __init__(self, 
                 reduction_type,
                 ndimensions):
        self.reduction_type = reduction_type # case-insensitive
        self.ndimensions = ndimensions
        
    def print_ndimensions(self):
        print("The number of dimensions is", ndimensions)
        
    def impute(self, X):
        imputer = SimpleImputer()
        self.data_imp_ = imputer.fit_transform(X)
    
    # fit and transform in either nmf or pca
    def fit(self, X):
        clf = self.reduction_type.lower();
        
        if clf == "nmf":
            self.clf_ = NMF(n_components=self.ndimensions, init='random', random_state=0)
        elif clf == "pca" :
            self.clf_ = PCA(n_components=self.ndimensions)
            
        self.model_ =  self.clf_.fit_transform(self.data_imp_)
        self.components_ = self.clf_.components_
       
        return self

            
    def reconstruct(self): 
       
        check_is_fitted(self, 'components_')
        self.recon_ = self.clf_.inverse_transform(self.model_)

    # takes aver values for each bundles 
    def plot_comparison(self):
        fig, ax = plt.subplots();
        mean = np.mean(self.recon_, axis = 0);
        data_mean = np.mean(self.data_imp_, axis = 0);
        ax.plot(mean);
        ax.plot(data_mean);
        
    def reconstruction_error(self): 

        loss = np.zeros(len(self.data_imp_));
        for i in range(len(self.data_imp_)):
             loss[i] = np.sqrt(np.mean(((self.recon_[i, :])-self.data_imp_[i, :])**2))
        return loss
       
        

In [ ]:
import afqinsight.datasets as ad
ad.__file__

In [ ]:
data = ad.load_afq_data(fn_nodes= "combined_tract_profiles.csv", fn_subjects="participant_data.tsv", 
                        unsupervised=True,return_bundle_means=False)

In [ ]:
dki_fa = data.X[:, 0:1800]


In [ ]:
data.X.shape

## cross-validation analysis

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from importlib import reload
import tools 
reload (tools)
from tools import crossvalidation

X = dki_fa;
loss = crossvalidation(X, PCA, 5);

In [ ]:
loss.shape



In [ ]:
import pandas as pd
import seaborn as sns

loss_df = pd.DataFrame(loss.T, columns = [2,4,8,16,32,64,128])


In [ ]:
loss_df


In [ ]:
pca_plot = sns.catplot( data=loss_df, kind='violin')
pca_plot.set_xlabels('number of dimensions')
pca_plot.set_ylabels('error in 5 folds croxx-validation')

In [ ]:
plt.hist(loss_df)

In [ ]:
#PCA for each bundle
X1 = np.reshape(X, (641, 18, 100))

In [ ]:
labels = ['ARC_L','ARC_R','ATR_L','ATR_R','CGC_L','CGC_R','CST_L','CST_R', 'FA','FP','IFO_L', 'IFO_R','ILF_L','ILF_R','SLF_L','SLF_R','UNC_L','UNC_R']

In [ ]:
def cvinbundle(bundle, dcom, num_fold):
    ind_bun = labels.index(bundle)
    arr= np.zeros((641, 100))
    arr = X1[:, ind_bun, :]
    loss = crossvalidation(arr, dcom, num_fold)
    loss_df_arc_l = pd.DataFrame(loss_arc_l.T, columns = [2,4,8,16,32,64])
    pca_plot = sns.catplot( data=loss_df_arc_l, kind='violin')
    pca_plot.set_xlabels('number of dimensions')
    pca_plot.set_ylabels('error in {} folds croxx-validation for {}'.format(num_fold, bundle))

In [ ]:
cvinbundle('IFO_L', NMF, 2)

In [ ]:
cvinbundle('IFO_L', PCA, 5)

In [ ]:
cvinbundle('ARC_L', PCA, 2)

In [ ]:
loss_arc_l= crossvalidation(arc_l, PCA, 2);
loss_df_arc_l = pd.DataFrame(loss_arc_l.T, columns = [2,4,8,16,32,64])

In [ ]:
pca_plot = sns.catplot( data=loss_df_arc_l, kind='violin')
pca_plot.set_xlabels('number of dimensions')
pca_plot.set_ylabels('error in 2 folds croxx-validation for arc_l')

In [ ]:
br = BundleReducer("pca", 3)

br.impute(arc_l)

In [ ]:
br.fit(X1[:, :, 0])
rec_arc_l = br.reconstruct()

In [ ]:
br.plot_comparison()

## PCA with 3 dimensions (dki_fa)

In [ ]:
br3 = BundleReducer("pca", 3)
br3.impute(dki_fa)
br3.fit(dki_fa)
rec_dki_fa = br3.reconstruct()

In [ ]:
br3.plot_comparison()

## PCA with 4 dimensions (dki_fa)

In [ ]:
br4 = BundleReducer("pca", 4)
br4.impute(dki_fa)
br4.fit(dki_fa)
rec_dki_fa = br4.reconstruct()

In [ ]:
br4.plot_comparison()

In [ ]:
error_br4 = br4.reconstruction_error()
np.mean(error_br4)

In [ ]:
plt.plot(br4.data_imp_[1, :])
plt.plot(br4.recon_[1, :])

In [ ]:
np.sqrt(np.mean((br4.data_imp_[1, :]-br4.recon_[1, :])**2)) # 4

In [ ]:
br5 = BundleReducer("pca", 5)
br5.impute(dki_fa)
br5.fit(dki_fa)
rec_dki_fa_5 = br5.reconstruct()

In [ ]:
# 6 dimensions
br6 = BundleReducer("pca", 6)
br6.impute(dki_fa)
br6.fit(dki_fa)
rec_dki_fa_6 = br6.reconstruct()

In [ ]:
# 7 dimensions
br7 = BundleReducer("pca", 7)
br7.impute(dki_fa)
br7.fit(dki_fa)
rec_dki_fa_7 = br7.reconstruct()

In [ ]:
# 8 dimension
br8 = BundleReducer("pca", 8)
br8.impute(dki_fa)
br8.fit(dki_fa)
rec_dki_fa_8 = br8.reconstruct()

In [ ]:
# 10 dimension
br10 = BundleReducer("pca", 10)
br10.impute(dki_fa)
br10.fit(dki_fa)
rec_dki_fa_10 = br10.reconstruct()

In [ ]:
def diff_bundles(data_imp, data_recon, sample):
    
    diff_bundle = np.zeros(18)
    for i in range(18):
        ind = 100*i;
        ind1 = ind +100;
        diff_bundle[i] = np.sqrt(np.mean((data_imp[sample, ind:ind1]-data_recon[sample, ind:ind1])**2))
        
    diff = np.zeros([641, 18])
    
    return diff_bundle

In [ ]:
diff_3 = np.zeros([641, 18])
for i in range(641):
    diff_3[i, :] = diff_bundles(br3.data_imp_, br3.recon_, i)
diff_3 = diff_3.mean(axis = 0)
diff_4 = np.zeros([641, 18])
for i in range(641):
    diff_4[i, :] = diff_bundles(br4.data_imp_, br4.recon_, i)
diff_4 = diff_4.mean(axis = 0)
diff_5 = np.zeros([641, 18])
for i in range(641):
    diff_5[i, :] = diff_bundles(br5.data_imp_, br5.recon_, i)
diff_5 = diff_5.mean(axis = 0)
diff_6 = np.zeros([641, 18])
for i in range(641):
    diff_6[i, :] = diff_bundles(br6.data_imp_, br6.recon_, i)
diff_6 = diff_6.mean(axis = 0)
diff_7 = np.zeros([641, 18])
for i in range(641):
    diff_7[i, :] = diff_bundles(br7.data_imp_, br7.recon_, i)
diff_7 = diff_7.mean(axis = 0)
diff_8 = np.zeros([641, 18])
for i in range(641):
    diff_8[i, :] = diff_bundles(br8.data_imp_, br8.recon_, i)
diff_8 = diff_8.mean(axis = 0)
diff_10 = np.zeros([641, 18])
for i in range(641):
    diff_10[i, :] = diff_bundles(br10.data_imp_, br10.recon_, i)
diff_10 = diff_10.mean(axis = 0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

data = np.random.randn(100)
n = 7;



p = sns.color_palette("rocket", n_colors=n)
# 2, 3, 5, 6, 8, 10 add legend
legend = [3, 4, 5, 6, 7, 8, 10]
x = range(0, 18)
labels = ['ARC_L',
 'ARC_R',
 'ATR_L',
 'ATR_R',
 'CGC_L',
 'CGC_R',
 'CST_L',
 'CST_R',
 'FA',
 'FP',
 'IFO_L',
 'IFO_R',
 'ILF_L',
 'ILF_R',
 'SLF_L',
 'SLF_R',
 'UNC_L',
 'UNC_R']
scaled_data = [diff_3.T, diff_4.T, diff_5.T, diff_6.T, diff_7.T, diff_8.T, diff_10.T]
for dd in range(n):
    plt.plot(scaled_data[dd], color=p[dd], label = legend[dd])
    plt.legend();
    plt.xticks(x, labels, rotation='vertical')
    plt.xlabel('bundles')